In [9]:
import os
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context

In [2]:
!pwd

/Users/vedantmahida/Desktop/end-to-end-ml-with-mlflow/notebooks


In [3]:
os.chdir('../')
!pwd

/Users/vedantmahida/Desktop/end-to-end-ml-with-mlflow


In [4]:
# 1. Update config.yaml [done]


# 3. entity
# entity = return type of a function
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  source_url: str
  local_data_file: Path
  unzip_dir: Path

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

# check import
CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH


(PosixPath('config/config.yaml'),
 PosixPath('params.yaml'),
 PosixPath('schema.yaml'))

In [6]:
# 4. Update the configuration manager in src cofig
class ConfigurationManager:
  def __init__(
      self,
      config_file_path = CONFIG_FILE_PATH,
      params_file_path = PARAMS_FILE_PATH,
      schema_file_path = SCHEMA_FILE_PATH
      ):
    
    # read yaml files
    self.config = read_yaml(config_file_path)
    self.params = read_yaml(params_file_path)
    self.schema = read_yaml(schema_file_path)

    # create directory (artifacts) from config.yaml
    create_directories([self.config.artifacts_root])
  
  
  def get_data_ingestion_config(self) -> DataIngestionConfig:
    # config.yaml
    config = self.config.data_ingestion 
    create_directories([config.root_dir])

    data_ingestion_config = DataIngestionConfig(
      root_dir=config.root_dir,
      source_url=config.source_url,
      local_data_file=config.local_data_file,
      unzip_dir=config.unzip_dir
    )

    return data_ingestion_config

In [7]:
# 5. Update components
import urllib.request as request
import zipfile
from mlProject.logging.customLogger import logger
from mlProject.utils.common import get_size

# Data Ingestion
class DataIngestion:
  def __init__(self, config: DataIngestionConfig):
    self.config = config
  
  def download_file(self):
    # download data file from src url to local file
    if not os.path.exists(self.config.local_data_file):
      filename, headers = request.urlretrieve(
        url=self.config.source_url, 
        filename=self.config.local_data_file
      )
      logger.info(f"{filename} downloaded with following info: \n{headers}")

    else:
      logger.info(f"{filename} already exists of size: {get_size(Path(self.config.local_data_file))}")
  

  def extract_zip_file(self):
    unzip_path = self.config.unzip_dir
    os.makedirs(unzip_path, exist_ok=True)
    # unzip files in local data file
    with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
      zip_ref.extractall(unzip_path)

In [10]:
# 6. Update pipeline
try:
  config = ConfigurationManager()
  data_ingestion_config = config.get_data_ingestion_config()
  data_ingestion = DataIngestion(config=data_ingestion_config)
  data_ingestion.download_file()
  data_ingestion.extract_zip_file()
  
except Exception as e:
  raise e

Mar 19 12:19:16 common:65 INFO YAML file: <_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfully!
Mar 19 12:19:16 common:65 INFO YAML file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully!
Mar 19 12:19:16 common:65 INFO YAML file: <_io.TextIOWrapper name='schema.yaml' mode='r' encoding='UTF-8'> loaded successfully!
Mar 19 12:19:16 common:29 INFO Directory created at: artifacts
Mar 19 12:19:16 common:29 INFO Directory created at: artifacts/data_ingestion
Mar 19 12:19:16 1081470741:19 INFO artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 